In [6]:
import os
import json
import pandas as pd
import joblib
import numpy as np
import socket
import matplotlib.pyplot as plt
import time
import subprocess
import tensorflow as tf
from keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.callbacks import ProgbarLogger, Callback
from sklearn.metrics import mean_squared_error
from io import StringIO
from threading import Thread, Event


2024-05-31 20:12:27.873117: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
file_path = "/home/francesco/Documents/Università/leCoseDiDario/AI Project/AI Project/src/data/2023.json"

with open(file_path, "r") as file:
        json_data = file.read()

data = json.loads(json_data)
  # Normalizzare i dati JSON annidati
header_df = pd.json_normalize(data["m_header"])
telemetry_df = pd.json_normalize(data["m_carTelemetryData"])
# Esplodere le colonne che contengono array
brakes_temperature_df = telemetry_df["m_brakesTemperature"].apply(pd.Series)
tyres_surface_temperature_df = telemetry_df["m_tyresSurfaceTemperature"].apply(pd.Series)
tyres_inner_temperature_df = telemetry_df["m_tyresInnerTemperature"].apply(pd.Series)
tyres_pressure_df = telemetry_df["m_tyresPressure"].apply(pd.Series)
surface_type_df = telemetry_df["m_surfaceType"].apply(pd.Series)
# Rinomina le colonne esplose
brakes_temperature_df.columns = [
    f"m_brakesTemperature_{i}" for i in range(brakes_temperature_df.shape[1])
]
tyres_surface_temperature_df.columns = [
    f"m_tyresSurfaceTemperature_{i}"
    for i in range(tyres_surface_temperature_df.shape[1])
]
tyres_inner_temperature_df.columns = [
    f"m_tyresInnerTemperature_{i}"
    for i in range(tyres_inner_temperature_df.shape[1])
]
tyres_pressure_df.columns = [
    f"m_tyresPressure_{i}" for i in range(tyres_pressure_df.shape[1])
]
surface_type_df.columns = [
    f"m_surfaceType_{i}" for i in range(surface_type_df.shape[1])
]
# Rimuovere le colonne originali dal DataFrame di telemetria
telemetry_df.drop(
    columns=[
        "m_brakesTemperature",
        "m_tyresSurfaceTemperature",
        "m_tyresInnerTemperature",
        "m_tyresPressure",
        "m_surfaceType",
    ],
    inplace=True,
)
# Combinare i dati esplosi con i dati di telemetria
telemetry_df = pd.concat(
    [
        telemetry_df,
        brakes_temperature_df,
        tyres_surface_temperature_df,
        tyres_inner_temperature_df,
        tyres_pressure_df,
        surface_type_df,
    ],
    axis=1,
)
# Combinare i dati header e telemetry
df = pd.concat([header_df] * len(telemetry_df), ignore_index=True)
df = pd.concat([df, telemetry_df], axis=1)
# Visualizzare i nomi delle colonne
print(df.columns)
# Selezionare le colonne rilevanti
target_columns = [
    "m_packetFormat",
    "m_gameYear",
    "m_gameMajorVersion",
    "m_gameMinorVersion",
    "m_packetVersion",
    "m_packetId",
    "m_sessionUID",
    "m_sessionTime",
    "m_frameIdentifier",
    "m_overallFrameIdentifier",
    "m_playerCarIndex",
    "m_secondaryPlayerCarIndex",
    "m_speed",
    "m_throttle",
    "m_steer",
    "m_brake",
    "m_clutch",
    "m_gear",
    "m_engineRPM",
    "m_drs",
    "m_revLightsPercent",
    "m_revLightsBitValue",
    "m_brakesTemperature_0",
    "m_brakesTemperature_1",
    "m_brakesTemperature_2",
    "m_brakesTemperature_3",
    "m_tyresSurfaceTemperature_0",
    "m_tyresSurfaceTemperature_1",
    "m_tyresSurfaceTemperature_2",
    "m_tyresSurfaceTemperature_3",
    "m_tyresInnerTemperature_0",
    "m_tyresInnerTemperature_1",
    "m_tyresInnerTemperature_2",
    "m_tyresInnerTemperature_3",
    "m_engineTemperature",
    "m_tyresPressure_0",
    "m_tyresPressure_1",
    "m_tyresPressure_2",
    "m_tyresPressure_3",
    "m_surfaceType_0",
    "m_surfaceType_1",
    "m_surfaceType_2",
    "m_surfaceType_3",
]
# Verifica che tutte le colonne esistano
missing_columns = [col for col in target_columns if col not in df.columns]
print("Missing columns:", missing_columns)
# Selezionare i dati per il training
target = df[target_columns]
# Espandi la dimensione del tuo input
X_train = np.expand_dims(target.values, axis=1)
# Convertire X_train in float32
X_train = X_train.astype("float32")
print(X_train.shape)
# Selezionare la colonna target (ad esempio, 'm_speed')
y_train = target["m_speed"].values
# Reshape di y_train per avere la forma corretta
y_train = y_train.reshape(-1, 1)
# Convertire y_train in float32
y_train = y_train.astype("float32")
# Verificare le forme dei dati
print("Shape di X_train:", X_train.shape)
print("Shape di y_train:", y_train.shape)
# Disabilita le GPU per evitare errori di memoria
print("Disabilitazione delle GPU...")
tf.config.experimental.set_visible_devices([], "GPU")
# Definisci il modello sequenziale
model = Sequential()
# Aggiungi il livello di input, specificando la forma (1, 43)
print("Aggiunta del livello di input...")
model.add(Input(shape=(1, 43)))
# Aggiungi un livello LSTM
print("Aggiunta del livello LSTM...")
model.add(LSTM(units=50, return_sequences=True))
# Aggiungi un altro livello LSTM se necessario
print("Aggiunta del secondo livello LSTM...")
model.add(LSTM(units=50))
# Aggiungi un livello denso per l'output
print("Aggiunta del livello denso per l'output...")
model.add(Dense(units=1))
# Compila il modello
print("Compilazione del modello...")
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
# Stampa il sommario del modello
print("Sommario del modello:")
model.summary()
# Definisci i callback
print("Definizione dei callback...")
callbacks = [ProgbarLogger()]
# Training del modello
print("Training del modello...")
model.fit(X_train, y_train, epochs=2, batch_size=32, callbacks=callbacks)
# Salva il modello
model.save("f1_deep_learning_model.keras")

Index(['m_packetFormat', 'm_gameYear', 'm_gameMajorVersion',
       'm_gameMinorVersion', 'm_packetVersion', 'm_packetId', 'm_sessionUID',
       'm_sessionTime', 'm_frameIdentifier', 'm_overallFrameIdentifier',
       'm_playerCarIndex', 'm_secondaryPlayerCarIndex', 'm_speed',
       'm_throttle', 'm_steer', 'm_brake', 'm_clutch', 'm_gear', 'm_engineRPM',
       'm_drs', 'm_revLightsPercent', 'm_revLightsBitValue',
       'm_engineTemperature', 'm_brakesTemperature_0', 'm_brakesTemperature_1',
       'm_brakesTemperature_2', 'm_brakesTemperature_3',
       'm_tyresSurfaceTemperature_0', 'm_tyresSurfaceTemperature_1',
       'm_tyresSurfaceTemperature_2', 'm_tyresSurfaceTemperature_3',
       'm_tyresInnerTemperature_0', 'm_tyresInnerTemperature_1',
       'm_tyresInnerTemperature_2', 'm_tyresInnerTemperature_3',
       'm_tyresPressure_0', 'm_tyresPressure_1', 'm_tyresPressure_2',
       'm_tyresPressure_3', 'm_surfaceType_0', 'm_surfaceType_1',
       'm_surfaceType_2', 'm_surfaceTyp

2024-05-31 20:14:24.105112: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-31 20:14:24.106067: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-31 20:14:24.107082: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Definizione dei callback...
Training del modello...
Epoch 1/2


2024-05-31 20:14:24.510825: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-31 20:14:24.512192: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-31 20:14:24.513348: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 2s 2s/sample - loss: 6066.1045 - mae: 35.9955
Epoch 2/2
1/1 [==============================] - 0s 6ms/sample - loss: 6064.1392 - mae: 36.0105
